In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("wine.data.txt", header = None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [3]:
header = pd.read_csv("Column_Names.txt")
header

,Type,Alcohol,Malic,Ash,Alcalinity,Magnesium,Phenols,Flavanoids,Nonflavanoids,Proanthocyanins,Color,Hue,Dilution,Proline


In [4]:
df.columns = header.columns
df.head()

,Type,Alcohol,Malic,Ash,Alcalinity,Magnesium,Phenols,Flavanoids,Nonflavanoids,Proanthocyanins,Color,Hue,Dilution,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [5]:
df.shape

(178, 14)

In [6]:
# Dependent variable distribution
df["Type"].value_counts()

2    71
1    59
3    48
Name: Type, dtype: int64

In [7]:
from sklearn.model_selection import train_test_split
trainDf, testDf = train_test_split(df, train_size=0.7, random_state = 123)

In [8]:
# Divide each dataset into Indep Vars and Dep var
depVar = 'Type'
trainX = trainDf.drop(depVar, axis = 1).copy()
trainY = trainDf[depVar].copy()
testX = testDf.drop(depVar, axis = 1).copy()
testY = testDf[depVar].copy()

print(trainX.shape)
testX.shape

(124, 13)


(54, 13)

### We will go through two types of models while tackling multi-class classification:

1. Models which can inherently perform multi-class classification, meaning the logic to perform multi-class classification is within the algorithm. You do not need to specify in the algorithm function either One-vs-One or One-vs-Rest. Example: Decision Tree, Random Forest, KNN
2. Models which need a specific mention of multi-class strategy, either One-vs-One or One-vs-Rest, in the algorithm function. Example: SVM, Multinomial Logistic Regression

Read more here: https://scikit-learn.org/stable/modules/multiclass.html


### 1A. Decision Tree Model

In [9]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
decisionTreeModel = DecisionTreeClassifier(random_state = 123).fit(trainX, trainY)
testPred = decisionTreeModel.predict(testX)

In [10]:
pd.crosstab(testY, testPred)

col_0,1,2,3
Type,,,
1,14,0,0
2,1,17,0
3,1,1,20


In [11]:
print(classification_report(testY, testPred))

              precision    recall  f1-score   support

           1       0.88      1.00      0.93        14
           2       0.94      0.94      0.94        18
           3       1.00      0.91      0.95        22

    accuracy                           0.94        54
   macro avg       0.94      0.95      0.94        54
weighted avg       0.95      0.94      0.94        54



### 1B. Decision Tree Model One-vs-One

In [12]:
from sklearn.multiclass import OneVsOneClassifier
decisionTreeModel_OneVsOne = OneVsOneClassifier(DecisionTreeClassifier(random_state = 123)).fit(trainX, trainY)
testPred = decisionTreeModel_OneVsOne.predict(testX)

In [13]:
pd.crosstab(testY, testPred)

col_0,1,2,3
Type,,,
1,14,0,0
2,0,16,2
3,0,6,16


In [14]:
print(classification_report(testY, testPred))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00        14
           2       0.73      0.89      0.80        18
           3       0.89      0.73      0.80        22

    accuracy                           0.85        54
   macro avg       0.87      0.87      0.87        54
weighted avg       0.86      0.85      0.85        54



### 1C. Decision Tree Model One-vs-Rest

In [15]:
from sklearn.multiclass import OneVsRestClassifier
decisionTreeModel_OneVsRest = OneVsRestClassifier(DecisionTreeClassifier(random_state = 123)).fit(trainX, trainY)
testPred = decisionTreeModel_OneVsRest.predict(testX)

In [16]:
pd.crosstab(testY, testPred)

col_0,1,2,3
Type,,,
1,13,1,0
2,1,16,1
3,2,0,20


In [17]:
print(classification_report(testY, testPred))

              precision    recall  f1-score   support

           1       0.81      0.93      0.87        14
           2       0.94      0.89      0.91        18
           3       0.95      0.91      0.93        22

    accuracy                           0.91        54
   macro avg       0.90      0.91      0.90        54
weighted avg       0.91      0.91      0.91        54



### 2A. SVM Model One-vs-One

In [18]:
from sklearn.svm import SVC

In [19]:
svmModel_OneVsOne = SVC(decision_function_shape="ovo", C = 100).fit(trainX, trainY)
testPred = svmModel_OneVsOne.predict(testX)

In [20]:
pd.crosstab(testY, testPred)

col_0,1,2,3
Type,,,
1,14,0,0
2,1,13,4
3,3,6,13


In [21]:
print(classification_report(testY, testPred))

              precision    recall  f1-score   support

           1       0.78      1.00      0.88        14
           2       0.68      0.72      0.70        18
           3       0.76      0.59      0.67        22

    accuracy                           0.74        54
   macro avg       0.74      0.77      0.75        54
weighted avg       0.74      0.74      0.73        54



### 2B. SVM Model One-vs-Rest

In [22]:
svmModel_OneVsRest = SVC(decision_function_shape="ovr", C = 75).fit(trainX, trainY)
testPred = svmModel_OneVsRest.predict(testX)

In [23]:
pd.crosstab(testY, testPred)

col_0,1,2,3
Type,,,
1,14,0,0
2,1,17,0
3,3,15,4


In [24]:
print(classification_report(testY, testPred))

              precision    recall  f1-score   support

           1       0.78      1.00      0.88        14
           2       0.53      0.94      0.68        18
           3       1.00      0.18      0.31        22

    accuracy                           0.65        54
   macro avg       0.77      0.71      0.62        54
weighted avg       0.79      0.65      0.58        54



##### Point to keep in mind: OneVsRestClassifier() or OneVsOneClassifier() are useful on algorithms that do NOT have the option to specify the strategy (OvsO or OvsR)

In [25]:
# Applying OneVsRestClassifier() or OneVsOneClassifier() on SVC() algorithm does NOT make much sense.
SVM_OneVsRest = OneVsRestClassifier(SVC(C=100)).fit(trainX, trainY)
testPred = SVM_OneVsRest.predict(testX)
print(classification_report(testY, testPred))

              precision    recall  f1-score   support

           1       0.82      1.00      0.90        14
           2       0.49      0.94      0.64        18
           3       1.00      0.09      0.17        22

    accuracy                           0.61        54
   macro avg       0.77      0.68      0.57        54
weighted avg       0.78      0.61      0.52        54

